In [40]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

# Load and preprocess data
raw_df = pd.read_csv('weatherAUS.csv')
raw_df.dropna(subset=['RainToday', 'RainTomorrow'], inplace=True)

# Create training, validation, and test sets
year = pd.to_datetime(raw_df.Date).dt.year
train_df, val_df, test_df = raw_df[year < 2015], raw_df[year == 2015], raw_df[year > 2015]

# Create inputs and targets
input_cols = list(train_df.columns)[1:-1]
target_col = 'RainTomorrow'
train_inputs, train_targets = train_df[input_cols], train_df[target_col]
val_inputs, val_targets = val_df[input_cols], val_df[target_col]
test_inputs, test_targets = test_df[input_cols], test_df[target_col]

# Identify numeric and categorical columns
numeric_cols = train_inputs.select_dtypes(include=np.number).columns.tolist()[:-1]
categorical_cols = train_inputs.select_dtypes(include='object').columns.tolist()

# Create preprocessing pipelines for both numeric and categorical data
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

# Combine transformers into a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Create a pipeline that includes preprocessing and the model
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, max_depth=8, random_state=42))
])

# Train the model
model_pipeline.fit(train_inputs, train_targets)

# Helper function to predict, compute accuracy & plot confusion matrix
def predict_and_plot(inputs, targets, name=''):
    preds = model_pipeline.predict(inputs)
    accuracy = accuracy_score(targets, preds)
    print(f"Accuracy {name}: {round(accuracy * 100,2)}%")
    return preds

# Evaluate on validation and test set
val_preds = predict_and_plot(val_inputs, val_targets, 'Validation')
test_preds = predict_and_plot(test_inputs, test_targets, 'Test')

# Save the trained model
joblib.dump(model_pipeline, 'aussie_rain_pipeline.joblib')

Accuracy Validation: 84.1%
Accuracy Test: 82.79%


['aussie_rain_pipeline.joblib']

In [66]:
test_inputs.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
count,25590.000000,25619.000000,25710.000000,10682.000000,7989.000000,24308.000000,25600.000000,24610.000000,25457.000000,23986.000000,23056.000000,23049.000000,14579.000000,12571.000000,25650.000000,24153.000000
mean,12.851344,23.869402,2.405053,6.146855,7.670009,39.316151,13.718867,18.232751,70.045646,52.078046,1017.240081,1014.828097,4.950065,4.933418,17.489283,22.171958
std,6.476907,7.311422,8.381634,4.963576,3.776338,13.497042,8.640277,8.530014,18.661433,20.762575,7.098359,7.016125,2.886161,2.767427,6.590811,7.095891
min,-7.800000,-4.800000,0.000000,0.000000,0.000000,7.000000,0.000000,0.000000,4.000000,3.000000,982.000000,977.100000,0.000000,0.000000,-7.200000,-5.400000
25%,8.100000,18.300000,0.000000,3.000000,5.000000,30.000000,7.000000,13.000000,58.000000,37.000000,1012.600000,1010.000000,2.000000,2.000000,12.700000,16.900000
50%,12.700000,23.400000,0.000000,5.400000,8.500000,37.000000,13.000000,17.000000,71.000000,53.000000,1017.200000,1014.900000,6.000000,6.000000,17.200000,21.800000
75%,17.600000,29.100000,0.800000,8.000000,10.700000,46.000000,19.000000,24.000000,84.000000,66.000000,1021.800000,1019.500000,8.000000,7.000000,22.100000,27.100000
max,31.400000,47.300000,225.000000,145.000000,14.100000,120.000000,130.000000,65.000000,100.000000,100.000000,1040.300000,1036.500000,8.000000,8.000000,37.600000,46.700000


In [68]:
categorical_cols = train_inputs.select_dtypes(include='object').columns.tolist()

In [70]:
categorical_cols

['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday']

In [86]:
train_inputs.Location.unique()

array(['Albury', 'BadgerysCreek', 'Cobar', 'CoffsHarbour', 'Moree',
       'Newcastle', 'NorahHead', 'NorfolkIsland', 'Penrith', 'Richmond',
       'Sydney', 'SydneyAirport', 'WaggaWagga', 'Williamtown',
       'Wollongong', 'Canberra', 'Tuggeranong', 'MountGinini', 'Ballarat',
       'Bendigo', 'Sale', 'MelbourneAirport', 'Melbourne', 'Mildura',
       'Nhil', 'Portland', 'Watsonia', 'Dartmoor', 'Brisbane', 'Cairns',
       'GoldCoast', 'Townsville', 'Adelaide', 'MountGambier', 'Nuriootpa',
       'Woomera', 'Albany', 'Witchcliffe', 'PearceRAAF', 'PerthAirport',
       'Perth', 'SalmonGums', 'Walpole', 'Hobart', 'Launceston',
       'AliceSprings', 'Darwin', 'Katherine', 'Uluru'], dtype=object)